In [1]:
import sys; sys.path.append('methods/')
from regridding import *

In [2]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [3]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [5]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("Westminster_Parliamentary_Constituencies_December_2019_Boundaries_UK_BUC")
sf

,pcon19cd,pcon19nm,bng_e,bng_n,long,lat,objectid,st_lengths,st_areasha,geometry
0,E14000530,Aldershot,484884,155126,-0.78410,51.288952,1,37241.703656,5.258835e+07,"POLYGON ((485406.902 159918.603, 486138.297 15..."
1,E14000531,Aldridge-Brownhills,404723,302568,-1.93166,52.620869,2,35258.924699,4.398713e+07,"POLYGON ((406519.098 305054.298, 405782.501 30..."
2,E14000532,Altrincham and Sale West,374132,389051,-2.39049,53.397659,3,40812.004306,5.087472e+07,"POLYGON ((379104.096 393143.903, 377127.904 39..."
3,E14000533,Amber Valley,440478,349675,-1.39770,53.042831,4,54421.854957,1.246901e+08,"POLYGON ((444868.402 353958.100, 444113.303 35..."
4,E14000534,Arundel and South Downs,510686,115542,-0.42635,50.928711,5,207778.021149,6.469497e+08,"POLYGON ((506641.497 128757.203, 506389.800 12..."
...,...,...,...,...,...,...,...,...,...,...
645,W07000076,Caerphilly,316860,188276,-3.20144,51.587181,646,63865.409578,1.150569e+08,"POLYGON ((315131.401 200637.498, 315308.299 19..."
646,W07000077,Islwyn,320517,196276,-3.15048,51.659618,647,59239.492373,1.117145e+08,"POLYGON ((322947.397 199706.204, 322974.804 19..."
647,W07000078,Vale of Glamorgan,302293,173081,-3.40742,51.448250,648,91342.706619,3.148419e+08,"POLYGON ((302260.803 179531.999, 302783.903 17..."
648,W07000079,Cardiff West,312437,178939,-3.26294,51.502571,649,36926.700752,5.081285e+07,"POLYGON ((312599.599 182852.798, 312692.900 18..."


In [8]:
sf_old = geopandas.read_file("Westminster")
sf_old

,NAME,AREA_CODE,DESCRIPTIO,FILE_NAME,NUMBER,NUMBER0,POLYGON_ID,UNIT_ID,CODE,HECTARES,AREA,TYPE_CODE,DESCRIPT0,TYPE_COD0,DESCRIPT1,geometry
0,Chorley Co Const,WMC,Westminster Constituency,CHORLEY_CO_CONST,1,1,124003,24758,E14000637,15520.886,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((358669.999 410970.300, 358665.802 41..."
1,City of Durham Co Const,WMC,Westminster Constituency,CITY_OF_DURHAM_CO_CONST,2,2,42609,25102,E14000641,18667.711,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((423763.701 535650.501, 423731.899 53..."
2,Colne Valley Co Const,WMC,Westminster Constituency,COLNE_VALLEY_CO_CONST,3,3,70205,24733,E14000645,18037.168,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((412070.003 419617.995, 412075.601 41..."
3,Darlington Boro Const,WMC,Westminster Constituency,DARLINGTON_BORO_CONST,4,4,70372,24974,E14000658,3003.961,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((427013.698 512595.800, 427039.703 51..."
4,Denton and Reddish Boro Const,WMC,Westminster Constituency,DENTON_AND_REDDISH_BORO_CONST,5,5,70275,24668,E14000661,2567.425,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((390121.601 397314.799, 390133.498 39..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,Burnley Boro Const,WMC,Westminster Constituency,BURNLEY_BORO_CONST,628,628,128717,24808,E14000609,11068.405,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((392703.401 434383.498, 392640.401 43..."
628,Bury North Boro Const,WMC,Westminster Constituency,BURY_NORTH_BORO_CONST,629,629,70209,24737,E14000611,5101.016,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((375025.397 414992.099, 375052.696 41..."
629,Bury South Boro Const,WMC,Westminster Constituency,BURY_SOUTH_BORO_CONST,630,630,124010,24715,E14000612,4844.616,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((375331.198 408381.601, 375331.997 40..."
630,Calder Valley Co Const,WMC,Westminster Constituency,CALDER_VALLEY_CO_CONST,631,631,70577,24779,E14000614,30626.295,0.0,VA,CIVIL VOTING AREA,None,None,"POLYGON ((388662.604 425190.800, 388678.303 42..."


In [9]:


# load the raw data
fnm = "../gridded_data/maxtmax_UKCP18-12km_ens-mean-maps_19801201-20101130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.maxtmax

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "objectid")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [12]:

# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.objectid[i]]).assign_coords(constituency = ("region", [sf.pcon19cd[i]])) for i in range(len(sf))], "region")

#~

In [13]:

# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [14]:
region_all=region_all.assign_coords(region=('name',sf.pcon19nm))

In [15]:
region_all.to_dataframe(name='value').reset_index().to_csv('maxtmax2049-2079.csv')

In [ ]:
#new variable- precipitation projections 



import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/pr/pr_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.pr

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [ ]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [ ]:
region_all.to_dataframe(name='value').reset_index().to_csv('pr_projection_2049-2079.csv')

In [ ]:
#new data mean daily temperature 


import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/tas/tas_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.tas

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [ ]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [ ]:
region_all.to_dataframe(name='value').reset_index().to_csv('tas_projection_2049-2079.csv')

In [ ]:
#new variable: number of dangerously hot days 


import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/ndhi/ndhi_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.ndhi

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [ ]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [ ]:
region_all.to_dataframe(name='value').reset_index().to_csv('ndhi_projection_2049-2079.csv')

In [ ]:
#new variable -- drought duration based on 6 month evapotranspiration index

import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/spei6dd/spei6dd_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.spei6dd

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [ ]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [ ]:
region_all.to_dataframe(name='value').reset_index().to_csv('drought_duration_projection_2049-2079.csv')

In [ ]:
#new variable--- max five day precipitation 
import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/rx5day/rx5day_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.rx5day

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [ ]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [ ]:
region_all.to_dataframe(name='value').reset_index().to_csv('max_5_day_precipitation_projection_2049-2079.csv')